In [1]:
# Imports Train
from tweet_data_label import train_data

693
Tweet 1
'This is my f**king country': Racist white woman arrested for attacking passengers on New York bus https://t.co/Rfm0jauh72
_____

Tweet 2
RT @mussie671: Freestyle #3, Fiasco type shiiii https://t.co/0GTJscWORW
_____

Tweet 3
What you think Cole? @JColeNC @JColeDirect https://t.co/oE4Ipvq6hL
_____

Tweet 4
⚡️ “Redskins dominate the Raiders on Sunday Night Football ”https://t.co/4VuHWOs1gX
DATE: Sunday Night 
_____

Tweet 5
RT @NWSSanJuan: 215PM FLASH FLOOD EMERGENCY for A Dam Failure in Isabela Municipality y Quebradillas Municipality in Puerto Rico... #prwx h…
NEIGHBORHOOD: Isabela Municipality NEIGHBORHOOD: Quebradillas Municipality 
TIME: 215PM 
_____

Tweet 6
Posted @chinatown on the corner of 7th &amp; D st. NW, PERFECT WEATHER TO SLIDE UP!!
ADDRESS: 7th &amp; D st. NW 

POSITIVE: Posted @chinatown on the corner of 
_____

Tweet 7
Good Morning DMV, we are @ChinaTown on the corner of 7th &amp; G st NW. Hand crafted burgers, endless toppings, Hand cut fries,  PULL UP !!!


_____

Tweet 305
Bonjour DC! We're off the streets today. Happy holiday everyone! Find us tomorrow at Union Station on Mass ave &amp; No… https://t.co/iJHje2CHsV

NEGATIVE
_____

Tweet 306
Bonjour DC! We're off the streets today. We'll be back tomorrow at Chevy Chase farmers market on Taylor &amp;Brookville… https://t.co/slEJ0lc1YW

NEGATIVE
_____

Tweet 307
Bonjour DC! We're off the streets today. It's time to go renew our licenses. We'll be back tomorrow at Chevy Chase… https://t.co/Mm5XksTy5L

NEGATIVE
_____

Tweet 308
Friendly Reminder! Don't eat at home tomorrow. Visit Curley's Q under tent 10am-5pm. Takoma Park Street Fest… https://t.co/5DePp5WWb3

NEGATIVE
_____

Tweet 309
RT @roamingrooster1: Hello Roaming Rooster Family! Tomorrow starting at 4pm, one of our trucks will be posted at the newly opened Smitty's…

NEGATIVE
_____

Tweet 310
RT @truckeroo: Tomorrow is the last Truckeroo of the season! Come out to celebrate the start of an amazing weekend. See you there!Trucker…

NEGA

_____

Tweet 498
15% off our entire menu SUPER BOWL SUNDAY!!Promo code TOUCHDOWN All caps.Online too @ https://t.co/KYvzFwIDwL https://t.co/PXjdzy3hrM
_____

Tweet 499
15% off our entire menu SUPER BOWL SUNDAY!!Promo code TOUCHDOWN All caps.Online too @ https://t.co/KYvzFwIDwL https://t.co/DChC0bzVig
_____

Tweet 500
301 881 4422 https://t.co/96uVpcr8vZ
_____

Tweet 501
RT @marcustewartjr: Y’all swear y’all be so deep choosing a pill like red. Give me $10M right now. I refuse to take the ACT again.
_____

Tweet 502
Superbowl sunday!! https://t.co/MY4ZElSerK
_____

Tweet 503
https://t.co/RJEsfnvzbc
_____

Tweet 504
https://t.co/TjzHWpFpIV
_____

Tweet 505
https://t.co/esgl3I4MWa
_____

Tweet 506
https://t.co/5NZiNJajPJ
_____

Tweet 507
https://t.co/6yLYQqSSbK
_____

Tweet 508
RT @CrabCab: New Fried Salmon Sandwich 🤤🤩 https://t.co/MzPtSwQV3d
_____

Tweet 509
https://t.co/0Eullb6wMo
_____

Tweet 510
RT @senorbennett: @CrabCab @TASH__P https://t.co/2kGQcEW4kJ
_____

Tweet 511
New Fried Sal

In [2]:
# Imports Dependencies
import spacy
import re
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight

import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

Using TensorFlow backend.
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/farrisismati/anaconda3/envs/nlp_course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

In [3]:
# Quick Validation on Tweet Data
def tweet_data_validate(data):
    print(f'Length: {len(data)}')

In [4]:
tweet_data_validate(train_data)

Length: 693


In [5]:
# Load up spACY library
nlp = spacy.load('en')

In [6]:
affirmation_map = {
    'POSITIVE': 0,
    'NEGATIVE': 1,
    'NULL': 2
}
    
# Organize Data Methods
def tweet_clean(text):
    lower = []
    text = text.replace('&amp;', 'and')

    for token in nlp(text):
        lower.append(token.text.lower())
            
    return lower

# Only target affirmation data
def organize_tweet_data(train_data):
    affirmation_map = {
        'POSITIVE': 0,
        'NEGATIVE': 1,
        'NULL': 1
    }
    tweet_sequences = []
    target_affirm = []

    # Organize tweets & train data into arrays
    for tweet_data in train_data:
        tweet_sequences.append(tweet_clean(tweet_data[0]))
        affirmation = tweet_data[1]['affirmation'][0][0] # Only first affirmation
        target_affirm.append(affirmation_map[affirmation])
        
    return [tweet_sequences, target_affirm]

#  Tokenization Methods
def tokenize_tweets(tokenizer, data):
    tokenizer.fit_on_texts(data)
    
    return tokenizer.texts_to_sequences(data)
    
def create_zeros_array(length):
    zeros_arr = []

    i = 0
    while i < length:
        zeros_arr.append(0)
        i += 1

    return zeros_arr

def find_max_tweet_len(data):
    max_len = 0

    # Find longest sequence, pad everything to its length
    for tweet in data:
        seq_len = len(tweet)
        if seq_len > max_len:
            max_len = seq_len

    return max_len

def pad_array(data, max_len):
    zeros_len = max_len - len(data)
    zeros_arr = create_zeros_array(zeros_len)

    return [*data,*zeros_arr]

In [7]:
# Tokenize & Pad Tweet Data
tweet_sequences, target_affirm = organize_tweet_data(train_data)

tokenizer = Tokenizer()
tokenized_data = tokenize_tweets(tokenizer, tweet_sequences)

max_len = find_max_tweet_len(tokenized_data)

X = [pad_array(data, max_len) for data in tokenized_data]

In [8]:
# Check size of tokenized vocab
vocab_size = len(tokenizer.index_word)

print(f'Vocab Size: {vocab_size}')
print(f'Target Affirm Length: {len(target_affirm)}')

Vocab Size: 2181
Target Affirm Length: 693


In [9]:
# Convert tokenized data to Numpy Array
X = np.array(X)
y = to_categorical(target_affirm)

In [32]:
# Validate Data
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

X shape: (693, 35)
y shape: (693, 2)


In [33]:
# Setup Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [40]:
# Validate Data
print('X_train')
print(f'X_train len: {len(X_train)}')
# print(X_train)
print('')
print('y_train')
print(f'y_train len: {len(y_train)}')
# print(y_train)

X_train
X_train len: 464

y_train
y_train len: 464


In [41]:
# Scale Train Data
def scale_data(X_train, x_test):
    scaler_object = MinMaxScaler()
    scaler_object.fit(X_train)
    
    scaled_X_train = scaler_object.transform(X_train)
    scaled_X_test = scaler_object.transform(X_test)
    return [scaled_X_train, scaled_X_test]

scaled_X_train, scaled_X_test = scale_data(X_train, X_test)

In [42]:
# Setup Training Model
def create_model():
    model = Sequential()
    model.add(Dense(12,input_dim=max_len,activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    
#     model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [43]:
model = create_model()

# Review Model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 12)                432       
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 18        
Total params: 554
Trainable params: 554
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Train - 12/28/20 %45.63
model.fit(scaled_X_train,y_train, epochs=100, batch_size=15,verbose=2, shuffle=True)

Epoch 1/100
 - 0s - loss: 0.6869 - accuracy: 0.5582
Epoch 2/100
 - 0s - loss: 0.6804 - accuracy: 0.5647
Epoch 3/100
 - 0s - loss: 0.6742 - accuracy: 0.5582
Epoch 4/100
 - 0s - loss: 0.6669 - accuracy: 0.5560
Epoch 5/100
 - 0s - loss: 0.6581 - accuracy: 0.5668
Epoch 6/100
 - 0s - loss: 0.6482 - accuracy: 0.5862
Epoch 7/100
 - 0s - loss: 0.6358 - accuracy: 0.6142
Epoch 8/100
 - 0s - loss: 0.6231 - accuracy: 0.6444
Epoch 9/100
 - 0s - loss: 0.6079 - accuracy: 0.6832
Epoch 10/100
 - 0s - loss: 0.5934 - accuracy: 0.7155
Epoch 11/100
 - 0s - loss: 0.5789 - accuracy: 0.7241
Epoch 12/100
 - 0s - loss: 0.5668 - accuracy: 0.7349
Epoch 13/100
 - 0s - loss: 0.5542 - accuracy: 0.7414
Epoch 14/100
 - 0s - loss: 0.5441 - accuracy: 0.7522
Epoch 15/100
 - 0s - loss: 0.5364 - accuracy: 0.7500
Epoch 16/100
 - 0s - loss: 0.5286 - accuracy: 0.7586
Epoch 17/100
 - 0s - loss: 0.5234 - accuracy: 0.7565
Epoch 18/100
 - 0s - loss: 0.5186 - accuracy: 0.7629
Epoch 19/100
 - 0s - loss: 0.5133 - accuracy: 0.7629
Ep

In [39]:
# Predictions
model.predict_classes(scaled_X_test)
predictions = model.predict_classes(scaled_X_test)

print('Results')
print(f'X : {predictions}')
print(f'Y : {y_test.argmax(axis=1)}\n')

print('Confusion Matrix')
print(print(f'Positive, Negative'))
print(f'{confusion_matrix(y_test.argmax(axis=1),predictions)}\n')

print('Classification Report')
print(classification_report(y_test.argmax(axis=1),predictions))

# 12/28/20 %42
print('Accuracy Score')
print(accuracy_score(y_test.argmax(axis=1),predictions))

Results
X : [0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0
 0 0 0 1 1 0 1 1 1 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 1 1 1 1 1
 1 0 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1
 0 1 1 1 1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0
 1 0 0 0 1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 0 1 0
 0 0 0 1 0 0 0]
Y : [1 0 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 1 0 1 1 1 1 0 0 1 0 0 1 1 0 1 0
 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 0
 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1
 0 0 1 1 0 0 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1
 0 0 0 1 1 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 1
 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 1 0 1 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0
 0 1 0 1 0 1 0]

Confusion Matrix
Positive, Negative
None
[[74 26]
 

In [31]:
tokenized_tweets = tokenize_tweets(tokenizer, ["Hi we are at chinatown and 5th today"])
predict_tweets = [pad_array(data, max_len) for data in tokenized_tweets]

scaler_object = MinMaxScaler()
scaler_object.fit(X_test)
scale_test = scaler_object.transform(predict_tweets)
    
prediction = model.predict_classes(scale_test)

print(prediction)

if prediction[0] == 0:
    print('Positive Affirmation')
else:
    print('Negative or no Affirmation')

# Add More data
# See accuracy

# Create NER model
# Combine this model with NER Model in an API to return data
# Use in Lambda func

[1]
Negative or no Affirmation
